# Rigid-body Transformations

In the previous section, we introduced the idea of two different coordinates systems we called *voxel-space* and *world-space*. In order to use both systems, we need to be able to convert between them. As such, we need to be able to take voxel coordinates and convert them into millimetres, as well as take millimetre coordinates and convert them into voxels.

Very generally, the ability to transform coordinates from one system into another can be expressed as

$$
\begin{align}
    b_x &= m_{11}a_x + m_{12}a_y + m_{13}a_z + m_{14} \\
    b_y &= m_{21}a_x + m_{22}a_y + m_{23}a_z + m_{24} \\
    b_z &= m_{31}a_x + m_{32}a_y + m_{33}a_z + m_{34} 
\end{align}
$$

where $a_x$, $a_y$, $a_z$ represent the original coordinates and $b_x$, $b_y$, $b_z$ represent the transformed coordinates. You might recognise each new coordinate as a *linear combination* of the old coordinates and some values called $m$. You might also recognise that this is therefore a system of three linear equations, which can be represented using matrices. As such, we can compactly represent this transformation in the form $\mathbf{b} = \mathbf{Ma}$, which we can expand to

$$
\begin{bmatrix}
    b_x \\
    b_y \\
    b_z \\
    1
\end{bmatrix}
=
\begin{bmatrix}
    m_{11} & m_{12} & m_{13} & m_{14} \\
    m_{21} & m_{22} & m_{23} & m_{24} \\
    m_{31} & m_{32} & m_{33} & m_{34} \\
    0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    a_x \\
    a_y \\
    a_z \\
    1
\end{bmatrix}.
$$

The matrix $\mathbf{M}$ therefore contains the 12 parameters of the transformation (noticing that we have to add an extra row to make the matrix multiplication work). Multiplying a vector containing our original coordinates by $\mathbf{M}$ will give us the coordinates transformed into the alternative space. We can also use $\mathbf{M}^{-1}$ to convert from the transformed coordinates back into the original coordinates using $\mathbf{a} = \mathbf{M}^{-1}\mathbf{b}$. 

## Affine Transformations
The parameters in the matrix $\mathbf{M}$ encode what is known as an *affine* transformation. The word [affine](https://en.wikipedia.org/wiki/Affine_transformation) means that parallel lines remain parallel after transformation. As such, operations such as *bending* are not allowed. Transformation can therefore only be constructed by combining:

- Translations
- Rotations
- Zooming/Scaling
- Shearing

Examples of each of these operations are shown in {numref}`affine-fig`.

```{figure} images/affine.png
---
width: 500px
name: affine-fig
---
Illustration of the operations that are allowed when forming an affine transformation.
```

To get a feel for affine transformations, you can play around with the example below. Move the sliders to change the transformation matrix, which is then used to change the coordinates of every pixel in the image.

<iframe align="" frameborder="0" height="800" scrolling="no" src="https://www.geogebra.org/material/iframe/id/jyskrsgy/width/800/height/800/border/888888/sfsb/true/smb/false/stb/false/stbh/false/ai/false/asb/false/sri/true/rc/false/ld/false/sdz/false/ctl/false" title="AffineTransform2D" width="800"></iframe>

Building an affine transformation within the matrix $\mathbf{M}$ can be achieved by multiplying separate matrices that encode translations, rotations, zooms and shearing. As such, the final matrix has the form

$$
\mathbf{M} = \mathbf{M}_{T} \times \mathbf{M}_{R} \times \mathbf{M}_{Z} \times \mathbf{M}_{S}.
$$

Each one of these individual matrices has a specific form. For instance, translating an image by $q_{x}$ units along the $x$-axis, $q_{y}$ units along the $y$-axis and $q_{z}$ units along the $z$-axis would result in

$$
\mathbf{M}_{T} = 
\begin{bmatrix}
1 & 0 & 0 & q_{x} \\
0 & 1 & 0 & q_{y} \\
0 & 0 & 1 & q_{z} \\
0 & 0 & 0 & 1
\end{bmatrix}.
$$

Similarly, a rotation of $q$ radians around either the $x$-axis (referred to as *pitch*), the $y$-axis (referred to as *roll*) or the $z$-axis (referred to as *yaw*) would result in

$$
\mathbf{M}_{R_{x}} = 
\begin{bmatrix}
1 & 0        & 0       & 0 \\
0 & \cos(q)  & \sin(q) & 0 \\
0 & -\sin(q) & \cos(q) & 0 \\
0 & 0        & 0       & 1
\end{bmatrix},\quad 
\mathbf{M}_{R_{y}} = 
\begin{bmatrix}
\cos(q)  & 0 & \sin(q) & 0 \\
0        & 1 & 0       & 0 \\
-\sin(q) & 0 & \cos(q) & 0 \\
0        & 0 & 0       & 1
\end{bmatrix},\quad 
\mathbf{M}_{R_{z}} = 
\begin{bmatrix}
\cos(q)  & \sin(q) & 0 & 0 \\
-\sin(q) & \cos(q) & 0 & 0 \\
0        & 0       & 1 & 0 \\
0        & 0       & 0 & 1
\end{bmatrix},
$$

with

$$
\mathbf{M}_{R} = \mathbf{M}_{R_{x}} \times \mathbf{M}_{R_{y}} \times \mathbf{M}_{R_{z}}.
$$

You do not need to worry about all the trigonometry here. The main point is about the *structure*, in that rotations are encoded in both the diagonal and off-diagonal rows corresponding to the axes we are *not* rotating around.

Zooms by a factor of $q_{x}$, $q_y$ and $q_z$ are encoded in the diagonal elements, where a negative zoom will *flip* the axis. This operation is also referred to as *scaling* and is encoded using

$$
\mathbf{M}_{Z} = 
\begin{bmatrix}
q_{x} & 0     & 0     & 0 \\
0     & q_{y} & 0     & 0 \\
0     & 0     & q_{z} & 0 \\
0     & 0     & 0     & 1
\end{bmatrix}.
$$

Finally, shears are encoded using parameters in the top three off-diagonal elements

$$
\mathbf{M}_{S} = 
\begin{bmatrix}
1 & q_{1} & q_{2} & 0 \\
0 & 1     & q_{3} & 0 \\
0 & 0     & 1     & 0 \\
0 & 0     & 0     & 1
\end{bmatrix}.
$$

Note that we are often interested in transformations that move the brain, without changing its general size or shape. These forms of transformation are known as *rigid-body* and correspond to a subset of affine transformations that *do not* include shearing. Furthermore, zooms are only allowed to use the same value of $q$ for each axis, such that the brain can be scaled up and down, but not stretched or squashed.

## Converting Voxel-space to World-space

Now that we have seen how affine transformations work, we can get back to the idea of converting between voxel-space and world-space. The key thing to understand is that this conversion is just an affine transformation, consisting of translating, rotating and scaling the axes so they represent millimetres instead of voxels. This process is illustrated in {numref}`v2w-affine-fig`.

```{figure} images/voxel-to-world-affine.png
---
width: 800px
name: v2w-affine-fig
---
Illustration of the affine transformation used to covert between voxel-space and world-space.
```

## Transforming an Image in World-space

## Transforming an Image in Voxel-space